In [1]:
import pandas as pd
import pickle
from datetime import datetime

In [19]:
with open('data/movimentacao_contas.pkl', 'rb') as f:
    contas = pickle.load(f)
contas

,data_cadastro,tipo,categoria_pagamento,subcategoria_pagamento,pago,conta,descricao,observacoes,id_transferencia,paciente,medico,forma_pagamento,convenio,servicos,origem
0,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,500.00,SANTANDER,SALDO INICIAL,Definição de saldo inicial para a conta SANTAN...,None,None,None,None,None,None,SISTEMA
1,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,154.34,BANESE,SALDO INICIAL,Definição de saldo inicial para a conta BANESE.,None,None,None,None,None,None,SISTEMA
2,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,11.36,C6,SALDO INICIAL,Definição de saldo inicial para a conta C6.,None,None,None,None,None,None,SISTEMA
3,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,478.14,CONTA PIX,SALDO INICIAL,Definição de saldo inicial para a conta CONTA ...,None,None,None,None,None,None,SISTEMA
4,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,216.63,DINHEIRO,SALDO INICIAL,Definição de saldo inicial para a conta DINHEIRO.,None,None,None,None,None,None,SISTEMA


In [14]:
with open('data/ipes_consolidado.pkl', 'rb') as f:
    ipes = pickle.load(f)
ipes

,data_cadastro,paciente,codigo,codigo_exame,descricao,medico,unidade,valor,origem_dados,convenio,tipo_procedimento,indice_paciente,id_pendencia,id_unico,data_consolidacao,origem,status_conciliacao
0,2025-08-01,CLODOMIR BARRETO COSTA,9846,0,ULTRASSONOGRAFIA PROSTATA,ISRAEL SANTOS MARCELO,<NA>,77.69,clinica,IPES,consulta_exame,2025-08-01_CLODOMIR BARRETO COSTA,PEND_000337,20250801_9846_0_77.69_clinica_mvk,2025-09-22 15:57:08.778802,ipes_consolidado,baixado
1,2025-08-01,HELOISA SANTOS OLIVEIRA,9836,0,MAPA,MAPA/HOLTER/ECG,<NA>,114.40,clinica,IPES,consulta_exame,2025-08-01_HELOISA SANTOS OLIVEIRA,PEND_000335,20250801_9836_0_114.4_clinica_nd2,2025-09-22 15:57:08.778802,ipes_consolidado,baixado
2,2025-08-01,JOSÉ BISPO DOS SANTOS ALVES,9835,0,"HOLTER, MAPA",MAPA/HOLTER/ECG,<NA>,211.32,clinica,IPES,consulta_exame,2025-08-01_JOSÉ BISPO DOS SANTOS ALVES,PEND_000334,20250801_9835_0_211.32_clinica_lw9,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
3,2025-08-01,LAURA DANIELLE DOS SANTOS SIQUEIRA ALMEIDA,8052,0,,,,49.79,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_LAURA DANIELLE DOS SANTOS SIQUEIRA ...,PEND_000407,20250801_8052_0_49.79_laboratorio_detalhado_j90,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
4,2025-08-01,LAURA DANIELLE DOS SANTOS SIQUEIRA ALMEIDA,8052,0,,,,20.95,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_LAURA DANIELLE DOS SANTOS SIQUEIRA ...,PEND_000412,20250801_8052_0_20.95_laboratorio_detalhado_k6b,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3453,2025-09-20,SAULO RODRIGUES REIS,8998,0,,,,0.00,laboratorio_detalhado,IPES,exame_laboratorio,2025-09-20_SAULO RODRIGUES REIS,PEND_003448,20250920_8998_0_0.0_laboratorio_detalhado_zx5,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
3454,2025-09-20,SAULO RODRIGUES REIS,8998,0,,,,24.46,laboratorio_detalhado,IPES,exame_laboratorio,2025-09-20_SAULO RODRIGUES REIS,PEND_003450,20250920_8998_0_24.46_laboratorio_detalhado_b4p,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
3455,2025-09-20,SAULO RODRIGUES REIS,8998,40307220,,,,11.90,laboratorio_detalhado,IPES,exame_laboratorio,2025-09-20_SAULO RODRIGUES REIS,PEND_003447,20250920_8998_40307220_11.9_laboratorio_detalh...,2025-09-22 15:57:08.778802,ipes_consolidado,pendente
3456,2025-09-20,SAULO RODRIGUES REIS,8998,40307255,,,,24.61,laboratorio_detalhado,IPES,exame_laboratorio,2025-09-20_SAULO RODRIGUES REIS,PEND_003449,20250920_8998_40307255_24.61_laboratorio_detal...,2025-09-22 15:57:08.778802,ipes_consolidado,pendente


In [15]:
with open('data/convenio_detalhado.pkl', 'rb') as f:
    conv = pickle.load(f)
conv

,data_cadastro,codigo,paciente,convenio,exame,Sexo,data_nascimento,Valor R$,Valor Final,Nome do Usuário,Descrição Exame,codigo_exame,valor,origem,data_importacao,indice_paciente,id_unico
0,2025-08-01,8033,NAGILSON DE JESUS SANTOS,IPES,HG,M,19/08/1966,"22,74","22,74",CINTIA REGINA LIMA SOUZA,HEMOGRAMA COMPLETO,40304361,9.69,convenios,2025-09-22 15:56:40.626922,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_9.69_convenios_xo1
1,2025-08-01,8033,NAGILSON DE JESUS SANTOS,IPES,GLI,M,19/08/1966,"22,74","22,74",CINTIA REGINA LIMA SOUZA,GLICOSE JEJUM,40302040,4.35,convenios,2025-09-22 15:56:40.626922,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_4.35_convenios_784
2,2025-08-01,8033,NAGILSON DE JESUS SANTOS,IPES,URE,M,19/08/1966,"22,74","22,74",CINTIA REGINA LIMA SOUZA,URÉIA,40302580,4.35,convenios,2025-09-22 15:56:40.626922,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_4.35_convenios_rxv
3,2025-08-01,8033,NAGILSON DE JESUS SANTOS,IPES,CREA,M,19/08/1966,"22,74","22,74",CINTIA REGINA LIMA SOUZA,CREATININA,40301630,4.35,convenios,2025-09-22 15:56:40.626922,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_4.35_convenios_4p1
148,2025-08-01,8052,LAURA DANIELLE DOS SANTOS SIQUEIRA ALMEIDA,IPES,GLI,F,28/04/1986,"361,43","361,43",CINTIA REGINA LIMA SOUZA,GLICOSE JEJUM,40302040,4.35,convenios,2025-09-22 15:56:40.626922,2025-08-01_LAURA DANIELLE DOS SANTOS SIQUEIRA ...,20250801_8052_4.35_convenios_yew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13147,2025-09-20,8999,ELISÂNGELA RODRIGUES DOS SANTOS,IPES,MX1,F,17/04/1981,"132,84","132,84",CINTIA REGINA LIMA SOUZA,IGE FUNGOS (MX1),40307255,24.61,convenios,2025-09-22 15:56:40.626922,2025-09-20_ELISÂNGELA RODRIGUES DOS SANTOS,20250920_8999_24.61_convenios_dvx
13148,2025-09-20,8999,ELISÂNGELA RODRIGUES DOS SANTOS,IPES,IGEPC,F,17/04/1981,"132,84","132,84",CINTIA REGINA LIMA SOUZA,IGE ESP. P/ CASPA DE CAO (E5),0,0.00,convenios,2025-09-22 15:56:40.626922,2025-09-20_ELISÂNGELA RODRIGUES DOS SANTOS,20250920_8999_0.0_convenios_h6a
13149,2025-09-20,8999,ELISÂNGELA RODRIGUES DOS SANTOS,IPES,E1,F,17/04/1981,"132,84","132,84",CINTIA REGINA LIMA SOUZA,IGE ESP - EPITELIO - CASPA DE GATO,40307255,24.61,convenios,2025-09-22 15:56:40.626922,2025-09-20_ELISÂNGELA RODRIGUES DOS SANTOS,20250920_8999_24.61_convenios_nax
13150,2025-09-20,8999,ELISÂNGELA RODRIGUES DOS SANTOS,IPES,D201,F,17/04/1981,"132,84","132,84",CINTIA REGINA LIMA SOUZA,IGE ESP - ÁCAROS - BLOMIA TROPICALI,0,24.46,convenios,2025-09-22 15:56:40.626922,2025-09-20_ELISÂNGELA RODRIGUES DOS SANTOS,20250920_8999_24.46_convenios_vo2


In [16]:
with open('data/convenio_ipes.pkl', 'rb') as f:
    contas = pickle.load(f)
contas

,data_cadastro,paciente,convenio,origem,valor,guia_operadora,beneficiario_codigo,senha,seq,tabela,procedimento_codigo,valor_exec,status_conciliacao,indice_paciente,descricao
0,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,CONVENIO_PDF,4.35,173541062,0001000312550073,1735410621,4,00,40301630,4.35,pendente,2025-08-01_NAGILSON DE JESUS SANTOS,CREATININA
1,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,CONVENIO_PDF,4.35,173541062,0001000312550073,1735410621,3,00,40302580,4.35,pendente,2025-08-01_NAGILSON DE JESUS SANTOS,UREIA
2,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,CONVENIO_PDF,4.35,173541062,0001000312550073,1735410621,2,00,40302040,4.35,pendente,2025-08-01_NAGILSON DE JESUS SANTOS,GLICOSE
3,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,CONVENIO_PDF,9.69,173541062,0001000312550073,1735410621,1,00,40304361,9.69,pendente,2025-08-01_NAGILSON DE JESUS SANTOS,HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAÇÕES...
4,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,CONVENIO_PDF,21.18,173541160,0001000312550073,1735411601,1,00,40101010,21.18,pendente,2025-08-01_NAGILSON DE JESUS SANTOS,ECG CONVENCIONAL DE ATE 12 DERIVACOES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,2025-08-30,LYLIAN MACIEL DOS ANJOS LIMA,IPES,CONVENIO_PDF,4.35,174647050,0001000673040078,1746470507,3,00,40302580,4.35,pendente,2025-08-30_LYLIAN MACIEL DOS ANJOS LIMA,UREIA
1956,2025-08-30,LYLIAN MACIEL DOS ANJOS LIMA,IPES,CONVENIO_PDF,36.79,174647050,0001000673040078,1746470507,2,00,40302733,36.79,pendente,2025-08-30_LYLIAN MACIEL DOS ANJOS LIMA,HEMOGLOBINA GLICADA (FRAÇÃO A1C)
1957,2025-08-30,LYLIAN MACIEL DOS ANJOS LIMA,IPES,CONVENIO_PDF,4.35,174647050,0001000673040078,1746470507,1,00,40302040,4.35,pendente,2025-08-30_LYLIAN MACIEL DOS ANJOS LIMA,GLICOSE
1958,2025-08-30,HEITOR SIQUEIRA GONÇALVES,IPES,CONVENIO_PDF,0.00,174669267,0003000413220093,1746692674,2,00,40301630,0.00,pendente,2025-08-30_HEITOR SIQUEIRA GONÇALVES,CREATININA


In [3]:
df_ipes[df_ipes['paciente'].str.contains('NAGILSON')]

,data_cadastro,paciente,codigo,codigo_exame,descricao,medico,unidade,valor,origem_dados,convenio,tipo_procedimento,indice_paciente,id_unico,data_consolidacao,origem
24,2025-08-01,NAGILSON DE JESUS SANTOS,9834,0,ELETROCARDIOGRAMA,MAPA/HOLTER/ECG,<NA>,21.18,clinica,IPES,consulta_exame,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_9834_0_21.18_clinica_dw1,2025-09-22 09:19:16.661946,ipes_consolidado
25,2025-08-01,NAGILSON DE JESUS SANTOS,8033,40301630,CREATININA,,REDE SANTA SAÚDE,4.35,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_40301630_4.35_laboratorio_detalh...,2025-09-22 09:19:16.661946,ipes_consolidado
26,2025-08-01,NAGILSON DE JESUS SANTOS,8033,40302040,GLICOSE JEJUM,,REDE SANTA SAÚDE,4.35,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_40302040_4.35_laboratorio_detalh...,2025-09-22 09:19:16.661946,ipes_consolidado
27,2025-08-01,NAGILSON DE JESUS SANTOS,8033,40302580,URÉIA,,REDE SANTA SAÚDE,4.35,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_40302580_4.35_laboratorio_detalh...,2025-09-22 09:19:16.661946,ipes_consolidado
28,2025-08-01,NAGILSON DE JESUS SANTOS,8033,40304361,HEMOGRAMA COMPLETO,,REDE SANTA SAÚDE,9.69,laboratorio_detalhado,IPES,exame_laboratorio,2025-08-01_NAGILSON DE JESUS SANTOS,20250801_8033_40304361_9.69_laboratorio_detalh...,2025-09-22 09:19:16.661946,ipes_consolidado


In [1]:
arquivo = 'relConvDetalhadoPorLinha_17_09_2025.xlsx'

In [9]:
# Lê o arquivo Excel
df = pd.read_excel(arquivo, header=5)

# Mapeamento de colunas
colunas_mapeamento = {
    'Data': 'data_cadastro',
    'Documento': 'documento',
    'Empresa': 'empresa',
    'Código': 'codigo',
    'Exame': 'exame',
    'Descrição': 'descricao',
    'Código Exame': 'codigo_exame',
    'Paciente': 'paciente',
    'Data Nascimento': 'data_nascimento',
    'Endereço': 'endereco',
    'Valor': 'valor',
    'Unidade': 'unidade',
    'Local': 'local',
    'Convênio': 'convenio',
    'Matricula': 'matricula'           
    
}


# Renomeia colunas
colunas_existentes = {k: v for k, v in colunas_mapeamento.items() if k in df.columns}
df = df.rename(columns=colunas_existentes)

# Converte coluna de valor (formato com vírgula para separador decimal)
def converter_valor_convenio(valor):
    """Converte valores do formato '123,45' para float 123.45"""
    if pd.isna(valor) or valor == '':
        return 0.0
    
    # Converte para string se não for
    valor_str = str(valor)
    
    # Se já é um número (float/int), retorna convertido
    try:
        if ',' not in valor_str and '.' not in valor_str:
            return float(valor_str)
        elif ',' in valor_str and '.' not in valor_str:
            # Formato brasileiro: vírgula como separador decimal
            return float(valor_str.replace(',', '.'))
        elif '.' in valor_str and ',' not in valor_str:
            # Formato americano: ponto como separador decimal
            return float(valor_str)
        else:
            # Formato com milhares e decimais (ex: 1.234,56 ou 1,234.56)
            if valor_str.rfind(',') > valor_str.rfind('.'):
                # Formato brasileiro: 1.234,56
                valor_limpo = valor_str.replace('.', '').replace(',', '.')
            else:
                # Formato americano: 1,234.56
                valor_limpo = valor_str.replace(',', '')
            return float(valor_limpo)
    except ValueError:
        return 0.0

if 'valor' in df.columns:
    df['valor'] = df['valor'].apply(converter_valor_convenio)

# Converte data
if 'data_cadastro' in df.columns:
    df['data_cadastro'] = pd.to_datetime(df['data_cadastro'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['data_cadastro'])

# Adiciona identificadores
df['origem'] = 'convenios'
df['data_importacao'] = datetime.now()

# GERAÇÃO DE ID ÚNICO
df['id_unico'] = (
    pd.to_datetime(df['data_cadastro']).dt.strftime('%Y%m%d') + '_' +
    df['codigo'].astype(str) + '_' +
    df['valor'].astype(str) + '_' +
    df['origem']
).apply(lambda x: x + '_' + gerar_id_aleatorio())
df

NameError: name 'gerar_id_aleatorio' is not defined

In [1]:
arquivo = 'CVS_010856404_20250908_20250912_00000000023420558.xlsx'

In [13]:
COLS_KEEP = [
        'Cartões', 'Data/Hora \nda Venda', "Data Prevista do 1º Pagamento", 
        "Descrição do Lançamento", "Total de Parcelas", "Valor Bruto",
        'Valor da Taxa \ne/ou Tarifa', "Valor Líquido"
    ]

In [23]:
from datetime import datetime
df = pd.read_excel(arquivo, sheet_name='ANALITICO', header=7, dtype=str)
df = df.dropna(subset=['Cód. Estabelecimento'])
df = df.dropna(axis=1, how='all')
# Exclui linhas onde a primeira coluna seja vazia
primeira_col = df.columns[0]
df = df[df[primeira_col].notna()]
df = df[df[primeira_col].astype(str).str.strip() != '']
# Mantém somente colunas alvo existentes
cols_exist = [c for c in COLS_KEEP if c in df.columns]
df = df[cols_exist].copy()
df.columns = ['cartoes','data_venda','data_prevista_1_pagamento','descricao_lancamento','n_parcelas','valor_bruto','valor_taxa','valor_liquido']
# Tratamento de datas
if "data_venda" in df.columns:
    # Converte para datetime e depois extrai apenas a data no formato brasileiro
    df["data_venda"] = pd.to_datetime(df["data_venda"])
    df["data_venda"] = df["data_venda"].dt.strftime('%d/%m/%Y')
df['origem'] = 'cartao_getnet'
df['data_importacao'] = datetime.now()
df['status'] = 'pendente'
# GERAÇÃO DE ID ÚNICO (adaptado para as novas colunas) - CORREÇÃO AQUI
# Converte valores para string tratando NaN
def safe_str(value):
    if pd.isna(value):
        return 'nan'
    return str(value)

df['id_unico'] = (
    pd.to_datetime(df["data_prevista_1_pagamento"], errors='coerce').dt.strftime('%Y%m%d').fillna('00000000') + '_' +
    df["valor_bruto"].apply(safe_str) + '_' +
    df["n_parcelas"].apply(safe_str) + '_' +
    df["valor_liquido"].apply(safe_str) + '_' +
    df['origem']
).apply(lambda x: x + '_' + gerar_id_aleatorio())
df

c:\Users\leopa\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NameError: name 'gerar_id_aleatorio' is not defined

In [ ]:
df['origem'] = 'cartao_getnet'
df['data_importacao'] = datetime.now()
df['status'] = 'pendente'

# GERAÇÃO DE ID ÚNICO (adaptado para as novas colunas) - CORREÇÃO AQUI
# Converte valores para string tratando NaN
def safe_str(value):
    if pd.isna(value):
        return 'nan'
    return str(value)

df['id_unico'] = (
    pd.to_datetime(df["data_prevista_1_pagamento"], errors='coerce').dt.strftime('%Y%m%d').fillna('00000000') + '_' +
    df["valor_bruto"].apply(safe_str) + '_' +
    df["n_parcelas"].apply(safe_str) + '_' +
    df["valor_liquido"].apply(safe_str) + '_' +
    df['origem']
).apply(lambda x: x + '_' + gerar_id_aleatorio())

In [2]:
with open('data/movimentacao_contas.pkl', 'rb') as f:
    df_movimento = pickle.load(f)
df_movimento

,data_cadastro,tipo,categoria_pagamento,subcategoria_pagamento,pago,conta,descricao,observacoes,id_transferencia,paciente,medico,forma_pagamento,convenio,servicos,origem
0,2025-01-01,SAIDA,SALDO INICIAL,AJUSTE,-3178.08,SANTANDER,SALDO INICIAL,Definição de saldo inicial para a conta SANTAN...,None,None,None,None,None,None,SISTEMA
1,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,0.33,BANESE,SALDO INICIAL,Definição de saldo inicial para a conta BANESE.,None,None,None,None,None,None,SISTEMA
2,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,11.36,C6,SALDO INICIAL,Definição de saldo inicial para a conta C6.,None,None,None,None,None,None,SISTEMA
3,2025-01-01,SAIDA,SALDO INICIAL,AJUSTE,-10091.82,CONTA PIX,SALDO INICIAL,Definição de saldo inicial para a conta CONTA ...,None,None,None,None,None,None,SISTEMA
4,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,260.86,DINHEIRO,SALDO INICIAL,Definição de saldo inicial para a conta DINHEIRO.,None,None,None,None,None,None,SISTEMA


In [4]:
df_movimento[df_movimento['categoria_pagamento']=='SALDO INICIAL'].to_pickle('data/movimentacao_contas.pkl')

In [33]:
with open('data/movimento_clinica.pkl', 'rb') as f:
    df_clinica = pickle.load(f)
with open('data/movimento_laboratorio.pkl', 'rb') as f:
    df_laboratorio = pickle.load(f)
with open('data/convenios_detalhados.pkl', 'rb') as f:
    df_convenios = pickle.load(f)

In [78]:
import os
from datetime import datetime
caminho_pendentes = 'data/recebimentos_pendentes.pkl'
novos_pendentes = []

# Processa pendências da Clínica
if df_clinica is not None and not df_clinica.empty and 'a_pagar' in df_clinica.columns:
    df_pend_clinica = df_clinica[df_clinica['a_pagar'] > 0].copy()
    if not df_pend_clinica.empty:
        df_pend_clinica.rename(columns={'a_pagar': 'valor_pendente', 'data_cadastro': 'data_operacao'}, inplace=True)
        df_pend_clinica['origem_recebimento'] = df_pend_clinica['convenio']
        novos_pendentes.append(df_pend_clinica[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente','origem']])

# Processa pendências do Laboratório
if df_laboratorio is not None and not df_laboratorio.empty and 'a_pagar' in df_laboratorio.columns:
    df_pend_lab = df_laboratorio[df_laboratorio['a_pagar'] > 0].copy()
    if not df_pend_lab.empty:
        df_pend_lab.rename(columns={'a_pagar': 'valor_pendente', 'data_cadastro': 'data_operacao'}, inplace=True)
        df_pend_lab['origem_recebimento'] = df_pend_lab['convenio']
        novos_pendentes.append(df_pend_lab[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente','origem']])

# Processa pendências de Convênios
if df_convenios is not None and not df_convenios.empty:
    df_pend_conv = df_convenios.copy()
    df_pend_conv.rename(columns={'valor': 'valor_pendente', 'data_cadastro': 'data_operacao', 'convenio': 'origem_recebimento'}, inplace=True)
    novos_pendentes.append(df_pend_conv[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente', 'origem']])

# Carrega os dados pendentes existentes
if os.path.exists(caminho_pendentes):
    df_pendentes_existente = pd.read_pickle(caminho_pendentes)
else:
    # Se o arquivo não existe, cria um DataFrame vazio para garantir a concatenação
    df_pendentes_existente = pd.DataFrame(columns=['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente', 'origem', 'id_pendencia'])

# Se houver novas pendências, processa e adiciona
if novos_pendentes:
    df_novos_pendentes = pd.concat(novos_pendentes, ignore_index=True)
    df_novos_pendentes['id_pendencia'] = [f"pend_{int(datetime.now().timestamp())}_{i}" for i in range(len(df_novos_pendentes))]
    
    # Concatena os dados existentes com os novos
    df_pendentes_atualizado = pd.concat([df_pendentes_existente, df_novos_pendentes], ignore_index=True)
else:
    # Se não houver novas pendências, o arquivo atualizado é o mesmo que o existente
    df_pendentes_atualizado = df_pendentes_existente

# # Remove duplicatas para garantir consistência e salva o arquivo final
df_pendentes_atualizado.drop_duplicates(subset=['data_operacao', 'paciente', 'valor_pendente', 'origem_recebimento', 'id_pendencia'], keep='last', inplace=True)
df_pendentes_atualizado.to_pickle(caminho_pendentes)

C:\Users\leopa\AppData\Local\Temp\ipykernel_9332\1610445931.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pendentes_atualizado = pd.concat([df_pendentes_existente, df_novos_pendentes], ignore_index=True)


In [79]:
df_pendentes_atualizado

,data_operacao,paciente,origem_recebimento,valor_pendente,origem,id_pendencia
0,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,21.18,clinica,pend_1754425212_0
1,2025-08-01,JOSÉ BISPO DOS SANTOS ALVES,IPES,211.32,clinica,pend_1754425212_1
2,2025-08-01,HELOISA SANTOS OLIVEIRA,IPES,114.40,clinica,pend_1754425212_2
3,2025-08-04,ZILMARIO VIEIRA SANTANA,IPES,21.18,clinica,pend_1754425212_3
4,2025-08-01,CLODOMIR BARRETO COSTA,IPES,77.69,clinica,pend_1754425212_4
...,...,...,...,...,...,...
6889,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,1.85,convenios,pend_1754425212_6889
6890,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,4.11,convenios,pend_1754425212_6890
6891,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,1.85,convenios,pend_1754425212_6891
6892,2025-08-04,JOSÉ ANTONIO DOS SANTOS,PARTICULAR,40.00,convenios,pend_1754425212_6892


In [ ]:

# Remove duplicatas para garantir consistência e salva o arquivo final
df_pendentes_atualizado.drop_duplicates(subset=['data_operacao', 'paciente', 'valor_pendente', 'origem_recebimento'], keep='last', inplace=True)
df_pendentes_atualizado.to_pickle(caminho_pendentes)

In [27]:
df = pd.read_html('relMovimentoDiarioDetalhado_04_08_2025 (1).xls')[0].iloc[5:-6]
df.columns = df.columns.get_level_values(-1)
df

,0,1,2,3,4,5,6,7,8,9,10,11
5,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,NaN
6,Cadastro,Unidade,Código,Paciente,Convenio,Total R$,Desc. R$,Acres. R$,Pago R$,Form. Pag.,Atendente,A Pag. R$
7,01/08/2025,RSS,0008036,BEATRIZ SANTOS BENIGNO,PARTICULAR,"R$ 12,00","R$ 0,00","R$ 0,00","R$ 12,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
8,01/08/2025,RSS,0008038,GEANE MARY SANTOS SANTANA,PARTICULAR,"R$ 13,20","R$ 0,20","R$ 0,00","R$ 13,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
9,01/08/2025,RSS,0008040,MARIA LUZIA DA CONCEIÇAO SANTOS,PARTICULAR,"R$ 146,20","R$ 0,00","R$ 0,00","R$ 146,20",Cartão de crédito,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
10,01/08/2025,RSS,0008045,JOEZIO SANTOS RESENDE,PARTICULAR,"R$ 102,00","R$ 7,00","R$ 0,00","R$ 95,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
11,01/08/2025,RSS,0008046,JULIANA DOS SANTOS RIBEIRO JESUS,PARTICULAR,"R$ 31,20","R$ 3,20","R$ 0,00","R$ 28,00",PIX JULIO,LUANNY,"R$ 0,00"
12,01/08/2025,RSS,0008047,ILDE COSME DOS SANTOS OLIVEIRA,PARTICULAR,"R$ 255,10","R$ 12,76","R$ 0,00","R$ 242,34",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"
13,01/08/2025,RSS,0008048,TAMIRES SANTOS FAUSTINO,LABORATORIO ASO,"R$ 14,00","R$ 0,00","R$ 0,00","R$ 14,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
14,01/08/2025,RSS,0008049,UERICA MACEDO SANTIAGO,PARTICULAR,"R$ 15,40","R$ 0,77","R$ 0,00","R$ 14,63",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"


In [8]:
df = pd.read_html('relMovimentoDiarioDetalhado_15_07_2025.xls', header=6)[0].iloc[:-5]
df

,Cadastro,Unidade,Código,Paciente,Convenio,Total R$,Desc. R$,Acres. R$,Pago R$,Form. Pag.,Atendente,A Pag. R$
0,01/07/2025,RSS,0007506,ELIZA FERREIRA DE OLIVEIRA,PARTICULAR,"R$ 37,60","R$ 1,60","R$ 0,00","R$ 27,00",Dinheiro,LUANNY,"R$ 0,00"
1,01/07/2025,RSS,0007506,ELIZA FERREIRA DE OLIVEIRA,PARTICULAR,"R$ 37,60","R$ 1,60","R$ 0,00","R$ 9,00",Dinheiro,LUANNY,"R$ 0,00"
2,01/07/2025,RSS,0007509,JOSE RAIMUNDO DOS SANTOS,PARTICULAR,"R$ 243,20","R$ 13,20","R$ 0,00","R$ 230,00",PIX JULIO,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
3,01/07/2025,RSS,0007511,RITA DE CASSIA SILVA FONSECA,PARTICULAR,"R$ 18,70","R$ 0,94","R$ 0,00","R$ 17,76",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"
4,02/07/2025,RSS,0007514,ADRIANA ALEMÃO SANTOS NOBRE,PARTICULAR,"R$ 13,20","R$ 0,00","R$ 0,80","R$ 14,00",PIX JULIO,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...
193,15/07/2025,RSS,0007688,JOSEFA LEITE SANTOS,SUS,"R$ 99,37","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 99,37"
194,15/07/2025,RSS,0007690,CLEOMISON ROSA SANTOS,SUS,"R$ 42,78","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 42,78"
195,15/07/2025,RSS,0007692,MARIA AUGUSTA ALVES DE OLIVEIRA,IPES,"R$ 336,23","R$ 0,00","R$ 0,00","R$ 0,00",NaN,LUANNY,"R$ 336,23"
196,15/07/2025,RSS,0007695,MARIA ELISA LIMA CALANZANI,IPES,"R$ 22,38","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 22,38"


In [10]:
df = pd.read_excel('relConvDetalhadoPorLinha_15_07_2025.xlsx', header=5)
df

,Data,Documento,Empresa,Código,Exame,Descrição,Código Exame,Paciente,Endereço,Valor,Unidade,Local,Convênio,Matricula
0,01/07/2025,NaN,NaN,7502,URE,URÉIA,020201069-4,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","1,85",REDE SANTA SAÚDE,NaN,SUS,NaN
1,01/07/2025,NaN,NaN,7502,TRI,TRIGLICERIDEOS,020201067-8,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","3,51",REDE SANTA SAÚDE,NaN,SUS,NaN
2,01/07/2025,NaN,NaN,7502,TGO,TRANSAMINASE OXALÁCETICA (TGO),020201064-3,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","2,01",REDE SANTA SAÚDE,NaN,SUS,NaN
3,01/07/2025,NaN,NaN,7502,TGP,TRANSAMINASE PIRUVICA (TGP),020201065-1,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","2,01",REDE SANTA SAÚDE,NaN,SUS,NaN
4,01/07/2025,NaN,NaN,7502,URI,URINA TIPO I,020205001-7,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","3,70",REDE SANTA SAÚDE,NaN,SUS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,15/07/2025,NaN,NaN,7695,PPF3,PARASITOLÓGICO DE FEZES - 3º AMOSTRA,40303110,MARIA ELISA LIMA CALANZANI,RUA AQUIDABA 222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2268,15/07/2025,NaN,NaN,7696,PPF2,PARASITOLÓGICO DE FEZES - 2º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2269,15/07/2025,NaN,NaN,7696,PPF1,PARASITOLÓGICO DE FEZES - 1º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2270,15/07/2025,NaN,NaN,7696,PPF3,PARASITOLÓGICO DE FEZES - 3º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
